# 阿里的 RAFT
1. Alpaca Intruction Dataset
2. query-generation: few-shots, query-type, 3 query examples, context chunk
2. prompt + qwen-110b/llama3-70b -> query-generation
3. answer-generation: prompt_tmpl, query, chunks(positive)
3. query,chunk,raw-answer + qwen-110b -> answer-rewritting -> advanced-answer
4. data synthesis: query, chunk, answer, retrieved negative chunks
4. query -> negatives retrieval -> retrieved negative chunks
4. advanced-answer, retrieved negative chunks, query, positive chunk -> data synthesis
5. sft instruction: prompt_tmpl, query, chunks(positive, negative)  -> output: reasoning_content, answer

# 关于阿里流程的疑问
1. answer-rewritting的作用是？提升是？为什么要重写答案？raw-answer和answer-rewritting有什么区别？raw-answer存在什么问题？
2. raw-answer是怎么来的？raw-answer是直接从chunk中抽取的答案，然后进行重写，重写后的答案作为最终的答案。
3. query generation的prompt_tmpl是什么？chunk,few shots(q_type,3 query examples)
# 对比现有的方法
1. query generation: prompt_tmpl/messages:[]
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""半导体专家，{num_questions}个问题。
        【核心要求】：问题设计准则，科学严谨性
        【禁止事项】
        【格式要求】
        论文开始{chunk_*4}论文结束"""
    }
]
2. cot_answer generation: prompt_tmpl/messages:[]
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""{
            "instruction": "你是半导体专家",
            "input": {
                "context": "{chunk_*4,chunk_id,source}",
                "question": "{question}"
            },
            "output": "{
                "answer": "根据切片信息中文作答。"
            }",
            "requirements": [
                "根据切片信息中文作答。"
            ]
        }"""
    }
]
3. data synthesis:
{
    "instruction": """{
        "instruction": "你是半导体专家",
        "input": {
            "context": "{chunk_*4,chunk_d4,chunk_id,source}",
            "question": "{question}"
        },
        "output": "{
            "answer": "根据切片信息中文作答。"
        }",
        "requirements": [
            "根据切片信息中文作答。"
        ]
    }""", 
    "input": "", 
    "output": <think>reasoning_content\n</think>\n\nanswer
}

# 当下的 RAFT 数据构造流程
1. 单篇文档
1. 获取单篇文档的 chunks
1. 获取单篇文档的 chunks 的 questions
1. 获取单篇文档的 questions 的 answers
1. 构建 RAFT 数据集

# 新版本 RAFT 数据集构建流程
1. 读取 文本源文件，分块，得到所有的文本块；
2. query generation  阶段，对每四个文本块进行 query 生成；
2. 在单篇文章中，选择连续4个文本块，对这四个文本块进行 query 生成；doc_id
2. 按照 文章为单位，进行遍历，对每篇文章/每四个文本块进行 query 生成；
3. answer generation 阶段，对每四个文本块/query进行 answer 生成；
3. [{chunk_*4, chunk_id, source}], question + prompt -> content, reasoning_content -> 推理数据；
4. 数据合成：RAFT数据集，sft数据集，推理数据集；
4. 推理数据集 instruction,input,output;
4. instruction -> context(chunk_*4+chunk_d4, chunk_id, source),question,prompt
4. output -> reasoning_content,answer

In [1]:
from utils.query_generation import read_file_names

In [2]:
filenames = read_file_names("data/", ext=".md")

len(file_names): 277


In [4]:
filenames[2]

'10大新显示技术_llm_correct.md'